# Title: Training ML Algorithms on Islet Isolation Data

### Abstract:
Islet isolations are a process by which the islet producing beta cells are isolated from the whole pancreas via enzyme digestion and high speed centrifugation.  The current network within the US for organ procurement is as follows, a hospital has a registered donor who has passed within the timeframe eligible for organ donation.  The hospital then contacts a partnered organ procurement organization which generally covers that geographical area, who then contacts potential processing centers such as ours as City of Hope to see whether or not we are interested in recieving the organs for research or transplant 
    
In order to ensure the proper use of the organ, various demographic information about the donor is released in a controlled manner through the proper channels so that processing centers have the knowledge and choice to reject donors if they do not have an applicable use.  Over time a somewhat anectodal judgement system for gauging potential donors has developed, such as rejecting donors who have high creatine levels, or donors with unknown downtime.  Although this anectodal information is based on indirect scientific studies of the pancreas [1] [2], these experimental analyses are not currently applied or provided prior to organ acceptance.  If a predictor could be made using more easily collectible information such as general bloodwork, demographic, or lifestyle information instead, it would be easier to integrate existing medical records to these algorithms.

### Research Question:
Is it possible to train a machine learning algorithm to predict islet isolation yield using only donor demographic characteristics?

### Hypothesis:
If the data is properly scaled an encoded so that the relevant categorical information can be properly valued by the machine learning algorithm, then it should be possible to train an algorithm to predict islet yield.  The accuracy of the algorithm may be limited by the sample size and data collection methods.

# Data Processing

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

#load excel sheet with data
data = pd.read_excel('Hu500++ isolation data.xlsx', skiprows=3, index_col = None, header = None)

In [2]:
data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,500,NaN,NaN,NaN,NaN,24,M,A(+),5.0,8.0,...,257050,83790.0,39884,381250.0,59400.0,32.596771,47.599952,1086.770428,517.302205,NaN
1,501,No,NaN,-3,Caucasian,56,F,O(+),5.0,6.0,...,388800,477667.0,322900,616000.0,563500.0,122.856739,67.599395,4854.339431,3281.504065,NaN
2,502,NaN,NaN,NaN,NaN,54,F,O(-),5.0,2.0,...,264710,285689.0,128783,103000.0,167500.0,107.925277,45.078039,4058.082386,1829.303977,NaN
3,503,NaN,NaN,NaN,NaN,51,F,O(+),5.0,5.0,...,242000,145633.0,172950,236000.0,285250.0,60.178926,118.757424,1408.442940,1672.630561,NaN
4,504,NaN,NaN,UW,Caucasian,46,F,O(+),5.0,8.0,...,543833,614200.0,386000,281250.0,412000.0,112.939082,62.845979,6786.740331,4265.193370,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,NaN,T2DM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,NaN,Transplant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
792,NaN,Not Processed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,NaN,Missing Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Start by creating named dictionary for index of raw data
categorical_cols = [4, 5, 6, 8, 9, 10, 11, 13, 19, 20, 21, 32]  # Indices of categorical columns
categorical_names = {
    4: 'Race',
    5: 'Age',
    6: 'Sex',
    8: 'Height Ft.',
    9: 'Height In.',
    10: 'Weight (lb)',
    11: 'BMI',
    13: 'HbA1c(%)',
    19: 'Pancreas weight(g)',
    20: 'Digested pancreas weight(g)',
    21: 'Switch time',
    32: 'Post Culture IEQ'
}
categorical_cols_to_encode = [4, 6]

In [4]:
#Scale data by calculating quartile distribution and replace original values with values 1-4 representing position on quartile distribution
#allows for comparison of variables and better processing by ML algorithm
import numpy as np

for col_index in categorical_cols:
    data.iloc[:, col_index] = pd.to_numeric(data.iloc[:, col_index], errors='coerce')

# Function to get quartile category
def get_quartile_category(value, quartiles):
    if pd.isnull(value):
        return 0
    elif value <= quartiles[0]:
        return 1
    elif value <= quartiles[1]:
        return 2
    elif value <= quartiles[2]:
        return 3
    else:
        return 4

# Loop through each categorical column
for col_index in categorical_cols:
    col_name = categorical_names[col_index]
    quartiles = np.nanpercentile(data.iloc[:, col_index], [25, 50, 75])
    data[col_name + '_Quartile'] = data.iloc[:, col_index].apply(lambda x: get_quartile_category(x, quartiles))

# Display the modified DataFrame
print(data)
print(data.shape)

       0                    1    2    3   4     5   6     7    8    9  ...  \
0    500                  NaN  NaN  NaN NaN  24.0 NaN  A(+)  5.0  8.0  ...   
1    501                   No  NaN   -3 NaN  56.0 NaN  O(+)  5.0  6.0  ...   
2    502                  NaN  NaN  NaN NaN  54.0 NaN  O(-)  5.0  2.0  ...   
3    503                  NaN  NaN  NaN NaN  51.0 NaN  O(+)  5.0  5.0  ...   
4    504                  NaN  NaN   UW NaN  46.0 NaN  O(+)  5.0  8.0  ...   
..   ...                  ...  ...  ...  ..   ...  ..   ...  ...  ...  ...   
790  NaN                 T2DM  NaN  NaN NaN   NaN NaN   NaN  NaN  NaN  ...   
791  NaN           Transplant  NaN  NaN NaN   NaN NaN   NaN  NaN  NaN  ...   
792  NaN        Not Processed  NaN  NaN NaN   NaN NaN   NaN  NaN  NaN  ...   
793  NaN  Missing Information  NaN  NaN NaN   NaN NaN   NaN  NaN  NaN  ...   
794  NaN                Atlas  NaN  NaN NaN   NaN NaN   NaN  NaN  NaN  ...   

     Sex_Quartile  Height Ft._Quartile  Height In._Quartile  \


C:\Users\Kevin\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


You can see the results of the quartile calculation appended as new rows to the end of the data object.  This means that the original data is still preserved in case we want to reference the specific cases while we can use the new rows as input into the machine learning algorithm.  This can be considered label encoding.

In [5]:
# Separate independent variables (features) and dependent variable (target)
X = data.iloc[:, 38:48]  # Selecting only the quartile calculated columns
y = data.iloc[:, 49]  # IEQ Yield
y = y.astype(np.int_)

# Clean target variable of categorical values like 'Yes' and 'No'
#label_encoder = LabelEncoder()
#y_encoded = label_encoder.fit_transform(y)

# One-hot encode specific categorical columns
X_to_encode = data.iloc[:, categorical_cols_to_encode]
encoder = OneHotEncoder(sparse=False, drop='first')
X_encoded = pd.get_dummies(X_to_encode, columns=X_to_encode.columns)

# Combine encoded columns with the rest of the categorical columns
new_index = [4,6]
X_remaining = X.drop(X.columns[new_index], axis=1)
X_concatenated = pd.concat([X_encoded, X_remaining], axis=1)

# Split the data into training, validation, and testing sets
X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.3, random_state=2)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=2)

In [6]:
# Check the shape of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Display the first few rows of X and y
print("Head of X:")
print(X.head())

print("\nHead of y:")
print(y.head())

Shape of X: (795, 10)
Shape of y: (795,)
Head of X:
   Race_Quartile  Age_Quartile  Sex_Quartile  Height Ft._Quartile  \
0              0             1             0                    1   
1              0             4             0                    1   
2              0             3             0                    1   
3              0             3             0                    1   
4              0             2             0                    1   

   Height In._Quartile  Weight (lb)_Quartile  BMI_Quartile  HbA1c(%)_Quartile  \
0                    3                     2             1                  2   
1                    2                     2             2                  1   
2                    1                     1             1                  3   
3                    2                     3             4                  1   
4                    3                     2             2                  1   

   Pancreas weight(g)_Quartile  Digested pancr

# Training the algorithms

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Define the hyperparameters grid to search through
param_grid = {
    'max_depth': [None, 1, 2, 3, 4],  # Vary the maximum depth of the tree
    'min_samples_split': [2, 3, 4, 5],  # Vary the minimum number of samples required to split an internal node
}

# Perform Grid Search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Print the best parameters found
print("Best Hyperparameters:")
print(best_params)

# Evaluate the best estimator on the test set
best_predictions = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test, best_predictions)
print(f"Accuracy of the best decision tree: {accuracy * 100:.2f}%")

Best Hyperparameters:
{'max_depth': 4, 'min_samples_split': 2}
Accuracy of the best decision tree: 29.17%


Since the dataset has a relatively low number of categorical variables and samples, the grid search will use smaller increments to instead search the more relevant combinations for accuracy and efficiency.  We can then use the calculated hyperparameters and compare the accuracy with some other trees.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth = 3, min_samples_split = 2, random_state=2)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Predict probabilities on the validation set
y_val_proba_clf = clf.predict_proba(X_val)  # Probabilities for each class

# Calculate ROC AUC score for the multiclass case
roc_auc_clf = roc_auc_score(y_val, y_val_proba_clf, multi_class='ovr', average='macro')

print(f"ROC AUC Score: {roc_auc_clf:.4f}")

The ROC AUC score of 0.5651 does not seems very high seeing as 1 shows a perfect fit.

In [ ]:
plt.figure(figsize=(15, 10))  # Adjust the figure size as needed
plot_tree(clf, filled=True, feature_names=X_concatenated.columns, class_names=True)
plt.title("Decision Tree Visualization")
plt.show()

In [ ]:
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth = 4, min_samples_split = 2, random_state=2)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Predict probabilities on the validation set
y_val_proba_clf1 = clf.predict_proba(X_val)  # Probabilities for each class

# Calculate ROC AUC score for the multiclass case
roc_auc_clf1 = roc_auc_score(y_val, y_val_proba_clf1, multi_class='ovr', average='macro')

print(f"ROC AUC Score: {roc_auc_clf1:.4f}")

The ROC AUC score of this tree is better at 0.61, however there would still be approximately 39% of samples being misclassified by this algorithm.

In [ ]:
plt.figure(figsize=(20, 15))  # Adjust the figure size as needed
plot_tree(clf, filled=True, feature_names=X_concatenated.columns, class_names=True)
plt.title("Decision Tree Visualization")
plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier

# Assuming you have X_train, X_test, y_train, y_test from the previous code

# Create an MLPClassifier model
mlp = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 50), max_iter=2000, random_state=2)

# Fit the model using the training set
mlp.fit(X_train, y_train)

# Predict probabilities on the validation set
y_val_proba = mlp.predict_proba(X_val)  # Probabilities for each class

# Calculate ROC AUC score for the multiclass case
# Replace y_true and y_score with actual y_val and y_val_proba from your dataset
roc_auc_mlp = roc_auc_score(y_val, y_val_proba, multi_class='ovr', average='macro')

print(f"ROC AUC Score: {roc_auc_mlp:.4f}")

The neural network appears to perform even worse, indicating that there may be a lack of sufficient information to predict the outcomes as the algorithm has converged but does not seem to be able to improve the loss function any further.

In [ ]:
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions

import warnings

# Suppress warnings
warnings.filterwarnings('ignore', message='You passed a edgecolor/edgecolors*')


# Assuming you have X_train, X_test, y_train, y_test from the previous code

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Choose two features to visualize decision boundaries
feature_combinations = list(combinations(X_train.columns, 2))  # Generate combinations of feature names

# Initialize PCA for dimensionality reduction (optional, if needed)
pca = PCA(n_components=2)  # You can adjust the number of components as needed

# Plot decision boundaries for each pair of features
for feat1, feat2 in feature_combinations:
    X_subset_train = X_train[[feat1, feat2]]
    X_subset_test = X_test[[feat1, feat2]]

    # Apply PCA for dimensionality reduction (optional)
    X_subset_train_pca = pca.fit_transform(X_subset_train)
    X_subset_test_pca = pca.transform(X_subset_test)

    # Fit the classifier on the subset of features
    mlp.fit(X_subset_train_pca, y_train_encoded)

    # Plot decision boundaries
    plt.figure(figsize=(8, 6))
    plot_decision_regions(X_subset_train_pca, y_train_encoded, clf=mlp, legend=2)

    # Adding axes annotations
    plt.xlabel(feat1)
    plt.ylabel(feat2)
    plt.title(f'Decision Boundaries for {feat1} vs {feat2}')

    # Highlighting test data
    plt.scatter(X_subset_test_pca[:, 0], X_subset_test_pca[:, 1], c=y_test, cmap='coolwarm', label='Test data')
    plt.legend()
    plt.show()


# Discussion
The decision boundaries for the neural network show that some of the categorical variables that I had included were not encoded in a way that appeared to be meaningful for the algorithm, like race and sex.  It is unlikely that these two variables would significantly improve the results of the classification however. 
Other intricacies like height being stored as two variables (ft. and in.) may have affected the decision boundary drawing, but when the graph itself is visualized, it appears that the groups were still somewhat accurately grouped together by relative height.
Overall the training of the algorithm shows results that are somewhat better than a random predicor (ROC_AUC score of 0.5), but there is definitely optimization which is missing which could further improve the accuracy of the algorithm.
More data relating to more islet isolations would also likely help improve the accuracy of the machine learning algorithms.

# Conclusion
I believe that my hypothesis is correct, and that it is indeed possible to predict islet yield based on donor demographic information plus some medical information that can be collected non-invasively.  However, the strength of correlation of these metrics is low, likely requiring a significantly larger dataset than the one I have provided along with a more strict and standard encoding method for the categorical variables like sex and gender so that they can be more accurately accounted for when being factored into the algorithm.

With the recent approval of a biological license for allogenic islet transplantation given to the University of Chicago, stronger predictive tools will likely be necessary to increase efficiency and reduce wait times for islet transplantation in the face of increased demand.  With only about 10 centers in the US performing islet isolations, it would be highly benificial to the process if the data from each center were merged and then further analysed.

References:

[1]    Komatsu, H., Qi, M., Gonzalez, N., Salgado, M., Medrano, L., Rawson, J., Orr, C., Omori, K., Isenberg, J. S., Kandeel, F., Mullen, Y., & Al-Abdullah, I. H. (2021). A Multiparametric Assessment of Human Islets Predicts Transplant Outcomes in Diabetic Mice. Journal Name, DOI: [10.1177/09636897211052291].

[2]    Wong, W. K. M., Jiang, G., Sørensen, A. E., Chew, Y. V., Lee-Maynard, C., Liuwantara, D., Williams, L., O’Connell, P. J., Dalgaard, L. T., Ma, R. C., Hawthorne, W. J., Joglekar, M. V., & Hardikar, A. A. (2019). The long noncoding RNA MALAT1 predicts human islet isolation quality. JCI Insight, DOI: [10.1172/jci.insight.129299].